In [1]:
import numpy as np 
import torch
import torchvision.models as models
import torchvision.datasets as dsets
import torch.nn as nn
from torch.utils.data import DataLoader
from model import Execution, FeatureExtractor

from tqdm import tqdm 

torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = False
torch.backends.cudnn.enabled = True
torch.backends.cudnn.allow_tf32 = False
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# init = np.zeros((64,3,7,7))
# Data = np.array(init, dtype=np.float16)
# print(Data.shape)


model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1).to('cuda')
#print(model) 

#verbose_resnet = 
#Execution(models.resnet50().to('cuda'))
#dummy_input = torch.ones(10,3,224,224)
# _ = verbose_resnet(dummy_input)

resnet_features = FeatureExtractor(model, layers = ["conv1","bn1","relu","maxpool","layer1","layer2","layer3","layer4","avgpool","fc"])
# features = resnet_features(dummy_input)

#print({name: output for name, output in features.items()})


for name, param in model.named_parameters():
     Data_shape = param.shape
     #FP16, FP32
     #print(param.view(torch.uint8).view(-1))
     
     Data_1d = param.view(-1)
     Data_1d_int = Data_1d.view(torch.uint8)
     
dataset = dsets.ImageFolder("/media/imagenet/val", models.ResNet50_Weights.IMAGENET1K_V1.transforms()) ### 2번째 인자, transform
loader = DataLoader(dataset= dataset, # dataset
                   batch_size=64,   # batch size power to 2
                   shuffle = False, # false
                   num_workers = 8, # num_workers 
                   pin_memory=True) # pin_memory 

correct = 0
total = 50000


model.eval()

for input, label in tqdm(loader):
    input = input.cuda(non_blocking=True)
    label = label.cuda(non_blocking=True) 
    output = resnet_features(input)
    
    #output = model(input)    
    pred = torch.argmax(output, 1)
    correct += (pred == label).int().sum()
acc1 = correct / total * 100

print(acc1)

/home/kkh/anaconda3/envs/1031_kkh/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 782/782 [00:48<00:00, 16.05it/s]


tensor(0.5500, device='cuda:0')


: 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cl1 = nn.Linear(25, 60)
        self.cl2 = nn.Linear(60, 16)
        self.fc1 = nn.Linear(16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


model = MyModel()
model.fc2.register_forward_hook(get_activation('fc2'))
x = torch.randn(1, 25)
output = model(x)
print(activation['fc2'])

tensor([[ 0.1130,  0.0874, -0.1723, -0.0508,  0.0868,  0.0694,  0.0226,  0.2247,
         -0.0240,  0.0072, -0.2114,  0.1505,  0.1794,  0.0358, -0.1231,  0.0392,
          0.0421,  0.0423,  0.0896,  0.0778,  0.0099, -0.0109,  0.0694,  0.0718,
          0.0120, -0.0319,  0.1469,  0.0471,  0.0532, -0.0952,  0.1329, -0.0057,
          0.0612, -0.0632,  0.1643, -0.0476, -0.0107, -0.1791, -0.0226, -0.0346,
         -0.1342,  0.0498,  0.1445,  0.0906,  0.0174, -0.1219, -0.1045, -0.0325,
         -0.0824,  0.2227, -0.0169,  0.0537, -0.0330,  0.0370,  0.0447, -0.0817,
         -0.0727,  0.0440, -0.1772, -0.0134, -0.0142, -0.0306, -0.1517,  0.0010,
         -0.0994,  0.1485, -0.1067,  0.0546,  0.2714,  0.1857, -0.0979, -0.3056,
          0.0518,  0.0942, -0.0892, -0.0907,  0.0079,  0.0082,  0.0886, -0.0502,
         -0.1103, -0.0112,  0.0397, -0.0341]])
